# Get-Started

Geemap tiene seis backends de trazado, incluidos *folium* , *ipyleaflet* , *plotly , pydeck , kepler.gl y heremap* . Un mapa interactivo creado utilizando uno de los backends de trazado se puede mostrar en un entorno Jupyter, como Google Colab, Jupyter Notebook y JupyterLab. De forma predeterminada, import geemaputilizará el ipyleafletmotor de trazado.

Los usuarios también pueden realizar análisis geoespaciales utilizando la GUI de WhiteboxTools con 468 herramientas de geoprocesamiento directamente dentro de la interfaz del mapa

El *ipyleaflet* paquete se basa en *ipywidgets* y permite la comunicación bidireccional entre el front-end y el backend, lo que permite el uso del mapa para capturar la entrada del usuario ( fuente ).

In [2]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.5 MB/s eta 0:00:00


Importación de librerias

In [3]:
import ee
import geemap

Generación de mapa

In [6]:
Map = geemap.Map(center=(19, -99), zoom=4)
Map

Map(center=[19, -99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [7]:
Map.add_basemap("Esri.WorldImagery")

Agregar mapas base de google

In [9]:
import os

os.environ["ROADMAP"] = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
os.environ["SATELLITE"] = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
os.environ["TERRAIN"] = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}'
os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

Map = geemap.Map()
Map.add_basemap("TERRAIN")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Agregar datos de Earth Engine

In [10]:
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")

Establecer parámetros de visualización

In [11]:
dem_vis = {
'min': 0,
'max': 4000,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

landsat_vis = {
    'min': 20,
    'max': 200,
    'bands': ['B4', 'B3', 'B2']
}

Mostrar datos en el mapa.

In [12]:
Map.addLayer(dem, dem_vis, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(landsat7, landsat_vis, 'Landsat 7')
Map.addLayer(states, {}, "US States")

Colecciones

In [13]:

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

In [14]:
print(collection.size().getInfo())

1233209


Obtener la primera imagen

In [15]:
image = collection.first()
geemap.image_props(image).getInfo()

{'CLOUD_COVER': 88.55,
 'CLOUD_COVER_LAND': 38.2,
 'EARTH_SUN_DISTANCE': 1.012707,
 'ESPA_VERSION': '2_23_0_1a',
 'GEOMETRIC_RMSE_MODEL': 7.666,
 'GEOMETRIC_RMSE_MODEL_X': 5.483,
 'GEOMETRIC_RMSE_MODEL_Y': 5.358,
 'IMAGE_DATE': '2014-05-24',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_001004_20140524_20170422_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1492857311000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2014-05-24T14:07:53.5175160Z',
 'SOLAR_AZIMUTH_ANGLE': -161.343674,
 'SOLAR_ZENITH_ANGLE': 57.854137,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 1,
 'WRS_ROW': 4,
 'system:asset_size': '633.527205 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_001004_20140524',
 'system:index': 'LC08_001004_20140524',
 'system:time_end': '2014-05-24 14:07:53',

In [16]:
Map.addLayer(image, {}, "First image")
Map.centerObject(image, 6)
Map

Map(bottom=59170.0, center=[78.11507273462493, -15.973500003273143], controls=(WidgetControl(options=['positio…

Filtro por fechas

In [17]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(
    '2020-01-01', '2020-12-31'
)

In [18]:
print(collection.size().getInfo())

140225


In [19]:
image2 = collection.first()
Map.addLayer(image2, {}, "Another image")
Map.centerObject(image2, 6)
Map

Map(bottom=3074.0, center=[78.14030281688883, -16.100684661875555], controls=(WidgetControl(options=['position…

Filtro por locación

In [20]:
roi = ee.Geometry.Point(-122.4488, 37.7589)

In [21]:
collection = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate('2020-01-01', '2020-12-31')
    .filterBounds(roi)
)

In [22]:
print(collection.size().getInfo())

21


In [23]:
Map = geemap.Map()
image = collection.first()

vis_param = {'min': 0, 'max': 2000, 'bands': ['B5', 'B4', 'B3'], 'gamma': 1.5}

Map.addLayer(image, vis_param, "First mage")
Map.centerObject(image, 8)

Map

Map(center=[37.47299148438048, -122.1059699711285], controls=(WidgetControl(options=['position', 'transparent_…

Filtro por metadatos

In [25]:
collection = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterDate('2020-01-01', '2020-12-31')
    .filterBounds(ee.Geometry.Point(-122.4488, 37.7589))
    .filterMetadata('CLOUD_COVER', 'less_than', 10)
    .sort("CLOUD_COVER")
)

In [26]:
Map = geemap.Map()
image = collection.first()

vis_param = {'min': 0, 'max': 2000, 'bands': ['B5', 'B4', 'B3'], 'gamma': 1.5}

Map.addLayer(image, vis_param, "First mage")
Map.centerObject(image, 8)

Map

Map(center=[37.47298074012687, -122.11977453809268], controls=(WidgetControl(options=['position', 'transparent…

Ontención de parametros

In [27]:
collection.aggregate_array('CLOUD_COVER').getInfo()

[0.07, 0.46, 0.55, 0.55, 2.96, 6.53]

In [28]:
collection.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C01/T1_SR/LC08_044034_20201012',
 'LANDSAT/LC08/C01/T1_SR/LC08_044034_20200403',
 'LANDSAT/LC08/C01/T1_SR/LC08_044034_20200302',
 'LANDSAT/LC08/C01/T1_SR/LC08_044034_20201129',
 'LANDSAT/LC08/C01/T1_SR/LC08_044034_20201028',
 'LANDSAT/LC08/C01/T1_SR/LC08_044034_20200606']

In [29]:
collection.aggregate_mean('CLOUD_COVER').getInfo()

1.8533333333333335

Uso del imagen reduce

In [30]:
median = collection.reduce(ee.Reducer.median())

Map.addLayer(
    median,
    {'bands': ['B4_median', 'B3_median', 'B2_median'], 'max': 0.3},
    'also median',
)
Map

Map(bottom=13074.0, center=[36.8268747428773, -119.89379882812501], controls=(WidgetControl(options=['position…